<H1> The punch you don’t see is the punch that hurts</H1>
 > heavyweight boxer Willie Pastrano (1935-1997)

# Overview

   Known potentially hazardous asteroids (PHAs) are tracked by a world-wide volunteer network of optical telescopes organized by ESA and JPL.  This PHAs observing service works for objects that are known and that due to combination of size, proximity and surface characteristics have a sufficiently high optical albedo to be optically detectable.  The analysis here presents an initial feasibility assessment of a radio frequency (RF) sensor network to detect unknown NEOs and PHAs as well as track known PHAs from the night side of earth (NSoE) using a reference signal from the day side of earth(DSoE).  One of the LibreSpace projects, SATNOGS, interconnects a world-wide network of satellite signal tracking enthusiasts with antennas of modest capabilities.  The analysis will pose and answer the question of what is the sized of a network of "enthusiast-grade" [1](#Footnotes) networked radio telescopes that could detect PHAs in the range not visible by ESA and JPL observing volunteers.  
   

# Background 

   The network would use DSoE and NSoE signals to develop passive radar track on NSoE near-earth objects (NEOs) that cross or reside outside of Earth Orbit. NEOs are classified in one of four classes according to their perihelion distance (q), aphelion distance (Q) and their semi-major axes (a): Atens, Apollos, Amors and Atiras[4]. The analysis here examines addresses the former three classes(note 1) because of their orbits location with respect to the earth.  The reference signal would be developed based on specific discrete solar RF radiation sources as isotropic radiators.

   notes:
   1. Atiras (a < 1 AU, Q < 0.983 AU) don't ever cross outside earth orbit and wouldn't be detectable through backscatter. Forward scatter investigation could complement this work at a later time.

# Geometry
   * <img src='astroid_detection_geometry.svg' width="600" height="400">

# Footnotes
A. <2 meter Az/El Rotator antenna, < 2Ghz Center Frequency, < 2 MHz BW,  < 2 Mbps Internet uplink, , <$2000 total cost, GPS Disciplined Oscillator 


# References
1. [H. D. Griffiths, Bistatic and Multistatic Radar](http://www.ittc.ku.edu/~callen/energy_harvesting/Griffiths2003pp9.pdf)
2. [M.K. Shepard et al., Radar observations and shape model of asteroid 16 Psyche, Icarus (2016)](http://dx.doi.org/10.1016/j.icarus.2016.08.011)
3. [Near Earth Objects - Dynamic Site](https://newton.spacedys.com/neodys/)
4. [Center for Near Earth Objects Website](https://cneos.jpl.nasa.gov/about/neo_groups.html)
5. [W. N. CHRISTIANSEN et al., The Distribution of Radio Brightness over the Solar Disat at a Wavelength of 21 Centimeters]()

In [3]:
import numpy as np

#System-wide Description
Terr = 50e-6           # seconds    
print(f"Multi-Antenna System Parameters: \n\tSite to Site Syncronization:{Terr * 1e6:.1f} microseconds")
print("\t\tstate of the art for 2022 using prosumer-grade \n\t\tGNSS/GPS TXCO master oscillator")

#Receiver Params
#Receiver System Calculations	Variable	Units
#RX antenna gain	GR	dBi
GR = 30                #dBi

#RX connector loss	LConR1	dB
LConR1 = 2             #dB

#RX cable loss	LCabR	dB
LCabR = 2             #dB

#RX connector loss (remote antenna)	LConR2	dB
LConR2 = 2             #dB

#RX Sensitivity  Prxs  dBm
Prxs = -115            #dBm

print(f"Antenna System Calcs: \n\tAntenna Gain: {GR} dBi")
print(f"\tCable & Connector Loss: {LConR1+LCabR+LConR2} dB")
print(f"\tRX Sensitivity  Prxs  {Prxs} dBm")

## SENSOR PARARMS
#Receiver Sensitivity Calculations	
#RX Noise Figure	NF
NF=5                    #dB

#Operating Temperature	T0
T0 = 290                #°K

#Effective Noise Temperature	Te
#  Te = T0(NF - 1)     
Te =T0*(np.power(10,(NF/10))-1)

#Boltzmann's constant	k
k=1.38E-23             #J/K

#Receive Bandwidth	BWRX
BWRX = 10              #MHz

#Baseline Length (meters)
BB = 50

#Rough Models for Active vs Quiet Sun returns Jansky's vs GHz
# Active Sun (freq in GHz) - returns NNNN Jy
def activeSun(freq : float)-> float:
    """returns active Sun Jy for a given freq in GHz"""
    return np.power(10.0,np.dot([np.power(np.log10(freq),ii) for ii in range(6)], np.array([7.575389683, -1.365882626, -0.3453708136, 0.8663449396, 0.1143669567, -0.1554750744])))
# Quiet Sun (freq in GHz) - returns NNNN Jy
def quietSun(freq : float)-> float:
    """returns quiet Sun Jy for a given freq in GHz"""
    return np.power(10.0,np.dot([np.power(np.log10(freq),ii) for ii in range(2)], np.array([5.44602420106103, 1.088711467])))
# Active Sun (freq in GHz) - returns NNNN Jy
def activeSundBmJy(freq : float)-> float:
    """returns active Sun dBm for a given freq in GHz"""
    return 10*np.dot([np.power(np.log10(freq),ii) for ii in range(6)], np.array([7.575389683, -1.365882626, -0.3453708136, 0.8663449396, 0.1143669567, -0.1554750744]))-230
# Quiet Sun (freq in GHz) - returns NNNN Jy
def quietSundBmJy(freq : float)-> float:
    """returns quiet Sun dBm for a given freq in GHz"""
    return 10.0*np.dot([np.power(np.log10(freq),ii) for ii in range(2)], np.array([5.44602420106103, 1.088711467]))-230

def spreadingLoss(freq : float, distance : float) -> float:
    """returns friis equation spreading loss in dB for a given freq in GHz and range in meters"""
    return 20 * (np.log10(freq/1000.0) + np.log10[distance]) - 27.55  # -(20*(np.log10(distance)+np.log10(freq))-27.55)

# Friis Equ for solar Radiation relative to 1 AU
#distAU is in solar orbitting range in AU
spreadingLossDBAU=lambda distAU: -10*np.log10(distAU)

#Antenna Temperature	TAnt
Tant = 300             #°k

#Noise Power (at RX)	Pn
# Pn = k(TAnt + Te)BWRX
Pn = k * (Tant +Te) * BWRX
Pn =10*np.log10(k*(Tant+Te)*(BWRX*1000000))+30

#Required Signal to Noise Ratio	SNRRX
SNRRX = Prxs-Pn

print(f"Rx Sensitivt Calcs:\n\tNoise Figure: {NF} dB")
print(f"\tTemperatures  Operating(To): {T0} °K  \tEffective(Te): {Te:.0f} °K  \tAntenna(Tant): {Tant} °K")
print(f"\tReceive Bandwidth	(BWRX): {BWRX} MHz \tNoise Power (at RX) : {Pn:.1f} dB")
print(f"\tRequired SNR: {SNRRX:.1f} dB")

#Passive Illumination Source

Multi-Antenna System Parameters: 
	Site to Site Syncronization:50.0 microseconds
		state of the art for 2022 using prosumer-grade 
		GNSS/GPS TXCO master oscillator
Antenna System Calcs: 
	Antenna Gain: 30 dBi
	Cable & Connector Loss: 6 dB
	RX Sensitivity  Prxs  -115 dBm
Rx Sensitivt Calcs:
	Noise Figure: 5 dB
	Temperatures  Operating(To): 290 °K  	Effective(Te): 627 °K  	Antenna(Tant): 300 °K
	Receive Bandwidth	(BWRX): 10 MHz 	Noise Power (at RX) : -98.9 dB
	Required SNR: -16.1 dB


In [3]:
# OC radar albedo  (sigmaHatOC)
# ratio of its OC radar cross section (σOC) to its cross-sectional area - See Ref 2
#                4σOC 
# sigmaHatOC = -----------
#               π Deff^2
# assume a conservative value of 0.1
sigmaHatOC=0.1


In [26]:
#Target Diameter (m)
Dt=20                 #m
albedo=0.15           #dimensionless
Tarea=np.pi*(Dt/2)**2  #sm
#TargetSize = Actual Area * Albedo 
SS=Tarea*albedo                 #sqm
dSS=10*np.log10(SS)
#Center Freq of Sample Signal
Fc=1.7               #GHz
#Speed of Light
c=299792458           #m/s
#Wavelength
Ld=c/(Fc*1e9)
#Incident Quiet Sun Flux on Target
# Fcenter - 
FLUXsunq=(Fc-BWRX/2/1000)*307555+16773
FLUXsunqq=quietSundBmJy(Fc)
#Effective Radiated Power Spectrum

#Effective Radiated Power over Rx BW

print(f"Target Diameter: {Dt} m  Albedo: {albedo}   Actual Area: {np.round(Tarea,2)} sqm")
print(f"Effective RCS dSS: {dSS:1.2f} dBsm")
print(f"Wavelength (Lambda - Ld): {Ld:1.3f} m")
print(f"quarter Wavelength (Lambda - Ld): {Ld/4:1.3f} m")
print(f"Quiet Sun Flex Incident power {FLUXsunq:5.0f} Jy  / {FLUXsunqq:5.0f} dBm")
print(f"Active Sun Flex Incident power {activeSun(Fc):5.0f} Jy  / {activeSundBmJy(Fc):5.0f} dBm")


Target Diameter: 20 m  Albedo: 0.15   Actual Area: 314.16 sqm
Effective RCS dSS: 16.73 dBsm
Wavelength (Lambda - Ld): 0.176 m
quarter Wavelength (Lambda - Ld): 0.044 m
Quiet Sun Flex Incident power 538079 Jy  /  -173 dBm
Active Sun Flex Incident power 17910436 Jy  /  -157 dBm


In [161]:
# Target size FOV



57.29577951308232

# Useful Quantities

 1. Irradiance Units [wikiPedia Ref Link](https://en.wikipedia.org/wiki/Irradiance)
    1. Spectral Radiance: watt per steradian per square metre per hertz 
        Radiance of a surface per unit frequency or wavelength. The latter is commonly measured in W⋅sr−1⋅m−2⋅nm−1. This is a directional quantity.
    2. Spectral intensity: watt per steradian per hertz 
        Radiant intensity per unit frequency or wavelength. The latter is commonly measured in W⋅sr−1⋅Hz-1. This is a directional quantity.  
2. Energy Units
   1. Jansky: unit of spectral flux density
        Pdbm*m^-2*Hz^-1 = 10*log10(Pjy)-230
        PdbW*m^-2*Hz^-1 = 10*log10(Pjy)-260
   2. erg/s to Watts: 1 W = 10000000 erg/s

# Random Supporting Links

1. [Asteroid Comparisons - Metal Ball Studios](https://www.youtube.com/watch?v=bSkPNMjRRio&t=54s)
2. [Nasa plan to save the earth from Asteroids](https://www.treehugger.com/neo-surveillance-killer-asteroids-nasa-4860013)
3. [Inferring asteroid surface properties from radar albedos and circular-polarization ratios](https://echo.jpl.nasa.gov/asteroids/virkki.etal.2014.radar.albedos.poln.ratios.pdf)

# Supporting Technologies

1. [High-accuracy Time and Frequency in VLBI](https://www.haystack.mit.edu/wp-content/uploads/2020/07/conf_TOW2019_seminar_Pazamickas_Sem.pdf)

/Users/leif/Documents/sinkt/SourceCode/Python/Space_and_Astronomy/NEA_Tracking_Calcs


***
$\mathbf{\text{Gradient Tree Boosting Algorithm}}$<br>
***
1.&emsp;Initialize model with a constant value $$f_{0}(x) = \textrm{arg min}_{\gamma} \sum \limits _{i=1} ^{N} L(y_{i}, \gamma)$$
2.&emsp;For m = 1 to M:<br>
&emsp;&emsp;(a)&emsp;For $i = 1,2,...,N$ compute<br>
    $$r_{im} = - \displaystyle \Bigg[\frac{\partial L(y_{i}, f(x_{i}))}{\partial f(x_{i})}\Bigg]_{f=f_{m−1}}$$
&emsp;&emsp;(b)&emsp;Fit a regression tree to the targets $r_{im}$ giving terminal regions<br>
&emsp;&emsp;&emsp;&emsp;$R_{jm}, j = 1, 2, . . . , J_{m}.$<br><br>
&emsp;&emsp;(c)&emsp;For $j = 1, 2, . . . , J_{m}$ compute<br>
$$\gamma_{jm} = \underset{\gamma}{\textrm{arg min}} \sum \limits _{x_{i} \in R_{jm}} L(y_{i}, f_{m−1}(x_{i}) + \gamma)$$
&emsp;&emsp;(d)&emsp;Update $f_{m}(x) = f_{m−1}(x) + \sum _{j=1} ^{J_{m}} \gamma_{jm} I(x \in R_{jm})$<br><br>
1. Output $\hat{f}(x) = f_{M}(x)$
***

In [12]:
#
# Conversion Rate 
# 1 ton of TNT = 4.184e9 J
#
mass=1000  #kg
V=14000    #MPH
V=14000 * 5280 / 3600 * 0.3048 # m/s

engeryPerKilo = 0.5 * V ** 2
print(f"Energy per kilogram of mass: {np.round(engeryPerKilo)}J {np.round(engeryPerKilo/4.184e6,2)} kg of TNT ")
print(f"Energy for full mass: {np.round(engeryPerKilo*mass)}J {np.round(engeryPerKilo*mass/4.184e9)} tons of TNT ")


Energy per kilogram of mass: 19584787.0J 4.68 kg of TNT 
Energy for full mass: 19584786637.0J 5.0 tons of TNT 
